In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
from scipy.stats.mstats import winsorize

In [40]:
#Ucitavamo podatke iz csv fajla i konvertujem churn varijablu u 0/1

data = pd.read_csv('/Users/djordjepetkovic/Downloads/task_data_churned.csv')
data['churned_status_int'] = data['churned_status'].replace({'No': 0, 'Yes': 1})

In [41]:
#Inicijalna provera podataka i vrednosti.

data.describe()

# Vecina varijabli su numericke i vecina imaju sve vrednosti. Dosta varijabli ima nule u vecini slucajeva, tako da
# ima smisla proveriti koliko su one bitne za predikciju i koja je njihova interpretacija. Takodje kvalitet podataka
# je nesto sto treba dodatno ispitati u smislu nedostajucih vrednosti i onda u skladu sa tim odluciti se za najbolji
# pristup u otklanjanju i/ili imputaciji tih nedostajucih vrednosti. Najbolje bi bilo kada bismo mogli da otkrijemo 
# razlog zbog koga nedostaju i onda imputiramo tacne vrednosti koje nedostaju. Osim toga mozemo da radimo i imputaciju
# na osnovu srednje vrednosti ili na osnovu vrednosti iz 'slicnih' instanci koji imaju datu vrednost koju zelimo
# da imputiramo. Npr ako u jednom redu nemamo vrednost za promenljivu 'action_gps_tracking', a u drugom redu
# koji je po ostalim promenljivim slican ovom prvom imamo vrednost za 'action_gps_tracking', onda prepisemo tu vrednost 
# u nedostajuci red.

# Jos jedna tema koju treba razmotriti je na koji nacin pristupiti outlierima. U zavisnosti od prirode samih
# podataka i znacenja odredjenih varijabli, mozemo drugacije pristupiti resavanju outliera. Jedan od methoda je tzv
# winsorizing, gde vrednost outliera zamenimo vrednoscu odredjenog percentila (npr 90%) te varijable. Time ne gubimo
# instance sa nedostajucim vrednostima i samim tim imamo vise podataka za treniranje modela.

# Ovaj dataset sadrzi i neke kategoricke varijable (country), tako da treba odluciti i o nacinu na koji cemo njih
# prebaciti u numericke. Jedan od nacina je prosta numeracija, tj stvaranje nove varijable, za svaku zemlju, koja ima
# vrednosti 0 ili 1. Druge metode ukljucuju npr postavljanje srednje vrednosti (mean encode) zavisne varijable (churn) 
# za odredjene grupe date kategoricke varijable. Npr za zemlju 'srbija' bismo stavili srednju vrednost zavisne 
# promenljive izracunatu na osnovu svih instanci koji imaju country='srbija'.

# S obzirom da vidimo da je odnos churn prema non churn u ovom data setu asimetrican, treba razmisliti i o metodama
# koje bismo mogli da pirmenimo da bismo resili tu asimetriju. Asimetricnost nije velika, ali svakako vredi ispitati 
# razlicite metode da bismo dosli do optimalnog resenja. Jedan od nacina na koji se ovo moze uraditi je primenom 
# smote algoritma, gde bismo generisali nove instance klase koja je manje pristuna i time izjednacili udeo te klase.
# Drugi nacin je postavljanje tezinskih parametara u samom algoritmu za treniranje modela.

# Da bismo izabrali najbolji model postoji vise faktora koje mozemo uzeti u obzir i onda testirati performanse razlicitih
# modela. Mozemo testirati razlicite algoritme, kao i razlicite pristupe u hendlovanju outliera, asimetricnih klasa
# nedostajucih vrednosti, itd i onda na osnovu toga videti sta u nasem konkretnom slucaju ima najvise smisla i daje
# najbolje rezultate.

# Za potrebe ovog zadatka ja sam se odlucio da uporedim dva modela, jedan osnovni model koji ce biti tzv baseline 
# i jedan unapredjeni gde cu primeniti neke tehnike na probleme opisane gore i onda uporediti performans.

# Treba jos naglasiti da je potrebno izabrati i statistiku koja ce nam biti cilj, tj koju pokusavamo da optimizujemo.
# U razlicitim slucajevima, u zavsnosti od potreba biznisa, to moze biti precision, recall, accuracy, itd..
# Ja cu se ovde skoncentrisati na jednu od njih (accuracy).

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,...,action_gps_tracking,action_screenshots,action_create_custom_field,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue,churned_status_int
count,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,...,876.000000,1044.000000,443.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000
mean,5.619504,0.827738,0.158273,28.043965,22.709432,0.383293,19.479616,0.175859,0.622702,8.494005,...,1.371005,1.417625,7.246050,61.286571,4.215827,5.728617,6.067946,0.257794,378.331825,0.319345
std,11.364130,3.527056,0.784527,80.761092,80.884964,3.089846,114.856050,0.380777,4.770705,52.699928,...,0.726969,0.791806,11.577418,85.179584,3.691711,4.893211,11.766325,2.957970,1007.971191,0.466316
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,38.961000,0.000000
50%,2.000000,0.000000,0.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,4.000000,24.000000,3.000000,4.000000,2.000000,0.000000,105.761500,0.000000
75%,6.000000,0.000000,0.000000,26.000000,15.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,2.000000,2.000000,7.000000,84.750000,7.000000,8.000000,6.000000,0.000000,333.459750,1.000000
max,206.000000,73.000000,20.000000,1923.000000,1740.000000,127.000000,3382.000000,1.000000,120.000000,1405.000000,...,8.000000,11.000000,106.000000,420.000000,14.000000,90.000000,215.000000,117.000000,27235.156000,1.000000


In [43]:
# Delimo dataset u X (nezavisne varijable) i y (zavisna varijable) vektore.

X = data.drop(['churned_status_int', 'churned_status'], axis=1)
y = data['churned_status_int']

# kreiramo dummy varijable za razlicite zemlje

X = pd.get_dummies(X)

# Nakon toga pravimo dva dataseta: jedan za treniranje podataka i drugi za testiranje.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definisemo xgboost model
clf = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Treniramo podatke na trening setu
clf.fit(X_train, y_train)

# Pravimo predikcije na trening i test podacima
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

# Evaluiramo model na trening i test podacima
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred)

# Gledamo bitnost varijabli u modelu
feature_importance = clf.feature_importances_

# Kreiramo data frame da stavimo bitnost varijabli
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importance
})

# Sortiramo data frame bitnosti
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)

print(accuracy_train)
print(accuracy_test)




                             Feature  Importance
6    action_time_entries_via_tracker    0.127062
19     value_number_of_active_months    0.053622
0                 ws_users_activated    0.028782
20         value_transactions_number    0.025955
87                     country_India    0.023774
..                               ...         ...
75          country_French Polynesia    0.000000
72                   country_Finland    0.000000
71                  country_Ethiopia    0.000000
70                   country_Estonia    0.000000
178                country_Wisconsin    0.000000

[179 rows x 2 columns]
0.9815092453773113
0.7325349301397206


In [36]:
# To su bili rezultati za baseline model. Sada cu odraditi neke tehnike opisane gore i istrenirati novi model i
# uporediti perfromans sa baseline modelom.

# Imputacija nedostajucih vrednosti

# S obzirom da postoje kolone sa vise od pola nedostajucih vrednosti ('action_create_custom_field' i 
# 'action_gps_tracking'), odlucujem da njih izbacim iz modela, a da u druge kolone koje imaju nedostajuce vrednosti 
# imputiram srednje vrednosti odgovarajuce kolone.

data_imputed = data.fillna(data.mean())

data_imputed = data_imputed.drop(['action_create_custom_field', 'action_gps_tracking'], axis=1)


# Ukljanjanje outliera: Koristicu winsorizing tehniku opisanu gore.

# Izdvajam numericke kolone

numeric_columns = data_imputed.select_dtypes(include=['number']).columns.difference(['churned_status_int'])

# Winsorize za svaku numericku kolonu

for column in numeric_columns:
    data_imputed[column] = winsorize(data_imputed[column], limits=(0.05, 0.05))  

# Nakon toga pravimo dva dataseta: jedan za treniranje podataka i drugi za testiranje.

data_train, data_test = train_test_split(data_imputed, test_size=0.2, random_state=42)

# Prvo cu odraditi Mean encoding za kategoricku varijablu country.

country_mean_encoding = data_train.groupby('country')['churned_status_int'].mean()
data_train['country_mean_encoded'] = data_train['country'].map(country_mean_encoding)
data_test['country_mean_encoded'] = data_test['country'].map(country_mean_encoding)


# Delimo dataset u X (nezavisne varijable) i y (zavisna varijable) vektore.

X_train = data_train.drop(['churned_status_int', 'churned_status'], axis=1)
y_train = data_train['churned_status_int']

X_test = data_test.drop(['churned_status_int', 'churned_status'], axis=1)
y_test = data_test['churned_status_int']


# Izbacujem kategoricku kolonu country

X_train = X_train.drop('country', axis=1)
X_test = X_test.drop('country', axis=1)


# kreiram parametre za asimetricne klase

class_weights = len(y_train) / (2 * np.bincount(y_train))

# Definisem xgboost model sa datim tezinama

clf = xgb.XGBClassifier(objective='binary:logistic', scale_pos_weight=class_weights[1], random_state=42)

# Treniram model

clf.fit(X_train, y_train)

# Pravim predikcije na trening i test podacima

y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

# Evaluacija modela

accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred)

print(accuracy_train)
print(accuracy_test)

0.9880059970014993
0.7405189620758483


/var/folders/4h/766zzr3x7td73m8sksdh2npw0000gn/T/ipykernel_70796/3878458521.py:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_imputed = data.fillna(data.mean())


In [37]:

# Gledamo bitnost varijabli u modelu

feature_importance = clf.feature_importances_

# Kreiramo data frame da stavimo bitnost varijabli

feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
})

# Sortiramo data frame

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)



feature_importance_df


,Feature,Importance
6,action_time_entries_via_tracker,0.251189
17,value_number_of_active_months,0.090734
0,ws_users_activated,0.058622
5,action_api_and_webhooks,0.045987
22,country_mean_encoded,0.044203
18,value_transactions_number,0.042763
10,action_lock_entries,0.042183
7,action_start_trial,0.040232
1,ws_users_deactivated,0.039408
16,value_days_to_purchase,0.034657


In [ ]:
# Zakljucne obzervacije: Vidimo da smo uspeli da podignemo accuracy za 1%. To nije veliko poboljsanje, ali definitivno
# pokazuje jedan od nacina na koji mozemo da poboljsamo model. 
# Accuracy od 74% nije los na test podacima, mada zeljena vrednost statistike koju gledamo da optimizujemo najvise
# zavisi od samog biznis problema i od toga sta definisemo unapred kao uspeh. 
# Da bismo dalje unapredili model, trebalo bi najvise raditi na dobavljanju jos podataka. Ako je moguce dobaviti
# jos samih korisnika, ali i varijabli koje se koriste za predikciju. Vidimo da je najznacajnija varijabla za model
# action_time_entries_via_tracker. Treba videti sta ona oznacava i onda u tom smeru ici sa dobavljanjem novih varijabli.
# U ovom procesu je najbitniji kontakt i feedback od biznisa, jer tu mozemo da saznamo i definisemo jos bitnih
# varijabli za ovaj konkretni problem, a i uvidimo kako oni uticu na samu predikciju.
# Jedan od potencijalnih problema koji se ovde vidi je i overfitting. Vdimo da postoji velika razlika izmedju accuracy
# na trening i test podacima. Postoje razlicite metode koje se bave resavanjem ovog problema, kao sto su npr
# redefinisanje modela na samo najbitnije varijable, i uz pomoc cross validacije provere koji od tih modela pokazuju
# priblizno slicne rezultate na trening i test podacima.

# Hyperparameter tuning je takodje jedna od tema kojoj se moze posvetiti vreme da bismo unapredili sam model, kroz
# izbor odgovarajucih hiperparametara za model.